# Building your first neural network in Python

## 1. Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

You’re hire as a Junior Data Scientist to work on a subscription streaming service. It is your first day in the job, you are setting up your stuff at your new desk, and suddenly, you get an email from Andrew Ng the Data Science lead.

<img src="../images/img_0136.png" width="600" height="700">

You got your first task and it is important for the future of the company, let's get into the code!

## 2. Get and transform the data

To begin, data preparation is crucial. In the real world, you'll typically retrieve data from a database. This often involves requesting permissions from DevOps teams and filing tickets with the data engineering team to obtain the necessary data. Once acquired, the data then needs to be preprocessed before it can be fed into your model.

For now, let's skip the data engineering part and just use a small dataset of 10 customers and convert it into tensors to use in our model.


In [2]:
# List of average monthly data usage (in GB) for 10 customers
avg_month_usage = [
    5.2,  # Customer 1: Low usage, might churn
    25.8, # Customer 2: High usage, likely to stay
    1.5,  # Customer 3: Very low usage, high churn risk
    18.7, # Customer 4: Moderate usage
    10.1, # Customer 5: Low-ish usage
    30.5, # Customer 6: Very high usage, loyal
    7.3,  # Customer 7: Low usage
    22.0, # Customer 8: Good usage
    3.9,  # Customer 9: Low usage, potential churn
    15.0  # Customer 10: Moderate usage
]

# List of churn status (0 = No Churn, 1 = Churn) for the same 10 customers
churn = [
    1,    # Customer 1: Churned
    0,    # Customer 2: Did not churn
    1,    # Customer 3: Churned
    0,    # Customer 4: Did not churn
    1,    # Customer 5: Churned
    0,    # Customer 6: Did not churn
    1,    # Customer 7: Churned
    0,    # Customer 8: Did not churn
    1,    # Customer 9: Churned
    0     # Customer 10: Did not churn
]

In [3]:
avg_month_usage_tensor = torch.tensor(avg_month_usage).float().reshape(-1, 1)
churn_tensor = torch.tensor(churn).float()

print(f"avg_month_usage_tensor:\n{avg_month_usage_tensor}\n\nchurn_tensor:\n{churn_tensor}")

avg_month_usage_tensor:
tensor([[ 5.2000],
        [25.8000],
        [ 1.5000],
        [18.7000],
        [10.1000],
        [30.5000],
        [ 7.3000],
        [22.0000],
        [ 3.9000],
        [15.0000]])

churn_tensor:
tensor([1., 0., 1., 0., 1., 0., 1., 0., 1., 0.])


Use the following code to create a ``TensorDataset`` and ``DataLoader`` for the data.

In [4]:
batch_size = 4 # Adjusted batch size for smaller dataset
num_samples = len(avg_month_usage_tensor)

# Create a TensorDataset and DataLoader
customer_dataset = TensorDataset(avg_month_usage_tensor, churn_tensor)
customer_dataloader = DataLoader(customer_dataset, batch_size=batch_size, shuffle=True)
print(f"Dummy DataLoader created with {len(customer_dataset)} samples.")

Dummy DataLoader created with 10 samples.


## 3. Intialize the model

We will be working with a binary classification problem, for this we need:

- Define the NN layer for output
- Select the loss function
- Set the optimizer

In [5]:
# --- 1. Define the Model (Single Layer) ---
# For a binary classification, the output layer typically has 1 neuron.
# We'll use a simple linear layer.
# The input_features should match the number of features in your dataset.
input_features = 1 

class BinaryClassifier(nn.Module):
    """
    A simple single-layer neural network for binary classification.
    """
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.linear = nn.Linear(input_dim, 1) # Single linear layer for output

    def forward(self, x):
        """
        Defines the forward pass of the model.

        Args:
            x (torch.Tensor): The input tensor.

        Returns:
            torch.Tensor: The output tensor (logits).
        """
        return self.linear(x)

model = BinaryClassifier(input_features)
print("Model defined:", model)

# --- 2. Define the Loss Function ---
# BCEWithLogitsLoss combines a Sigmoid layer and the BCELoss in one single class.
# This version is more numerically stable than using a plain Sigmoid followed by BCELoss.
criterion = nn.BCEWithLogitsLoss()
print("Loss function defined:", criterion)

# --- 3. Define the Optimizer ---
# Stochastic Gradient Descent (SGD)
learning_rate = 0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
print("Optimizer defined:", optimizer)


Model defined: BinaryClassifier(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)
Loss function defined: BCEWithLogitsLoss()
Optimizer defined: SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


## 4. Training Loop

This is where you will get hands-on the model, you need to define everything learned to build the training loop:

- Reset the gradients
- Forward pass
- Calculate loss
- Backward pass and optimize

In [16]:
def train_binary_classifier(model, dataloader, criterion, optimizer, num_epochs=10):
    """
    Performs the training loop for a binary classification model.

    Args:
        model (torch.nn.Module): The PyTorch model to be trained.
        dataloader (torch.utils.data.DataLoader): DataLoader providing training data.
        criterion (torch.nn.Module): The loss function (e.g., nn.BCEWithLogitsLoss).
        optimizer (torch.optim.Optimizer): The optimizer (e.g., optim.SGD).
        num_epochs (int): The number of epochs to train the model.
    """
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            # Zero the parameter gradients
            optimizer.zero_grad() # omit

            # Forward pass
            outputs = model(inputs)

            # Calculate loss
            # Ensure labels are float and have the correct shape (e.g., [batch_size, 1])
            loss = criterion(outputs, labels.float().unsqueeze(1)) # omit

            # Backward pass and optimize
            loss.backward() # omit
            optimizer.step() # omit

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

In [17]:
from unittests import test_train_binary_classifier

test_train_binary_classifier(train_binary_classifier)

Epoch 1/2, Loss: 0.7107
Epoch 2/2, Loss: 0.7107
❌ Function call test failed: step was not called.


In [ ]:
# --- 6. Run the Training Loop ---
print("\nStarting training...")
train_binary_classifier(model, customer_dataloader, criterion, optimizer, num_epochs=10) # Increased epochs for small dataset
print("Training finished.")

Conclusion:

You've run your first model and now have some insights into how things work, including the key components you can't miss in a training pipeline using PyTorch. You're ready to get back to Andrew and ask for more challenges. Let's head on to the next lesson to see more about your upcoming task.